UK trade in services: service type by partner country, non-seasonally adjusted

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/' + \
                  'internationaltrade/datasets/uktradeinservicesservicetypebypartnercountrynonseasonallyadjusted')

In [2]:
tab = scraper.distribution().as_pandas(header=None)
tab.rename(columns=tab.iloc[0], inplace=True)
tab = tab.iloc[1:, :]
tab = tab.drop(['Service type','Country'], axis = 1)

In [3]:
tab.columns.values[0] = 'Flow'
tab.columns.values[1] = 'Pink Book Services'
tab.columns.values[2] = 'ONS Partner Geography'

In [4]:
new_table = pd.melt(tab, id_vars=['Flow','Pink Book Services','ONS Partner Geography'], var_name='Period', value_name='Value')

In [5]:
new_table['Period'] = 'quarter/' + new_table['Period'].astype(str).str[0:4]+ '-' +   new_table['Period'].astype(str).str[-2:]              

In [6]:
new_table['Flow'] = new_table['Flow'].map(lambda s: s.lower().strip())

In [7]:
new_table['Seasonal Adjustment'] =  'NSA'
new_table['Measure Type'] =  'GBP Total'
new_table['Unit'] =  'gbp-million'

In [8]:
def user_perc(x):
    
    if (str(x) ==  '-') : 
        return 'itis-nil'
    elif ((str(x) == '..')):
        return 'disclosive'
    else:
        return None
    
new_table['Marker'] = new_table.apply(lambda row: user_perc(row['Value']), axis = 1)

In [9]:
new_table['Value'] = pd.to_numeric(new_table['Value'], errors = 'coerce')

In [10]:
new_table['Period'] = new_table['Period'].map(
    lambda x: {
        'quarter/2016-16' : 'year/2016', 
        'quarter/2017-17' : 'year/2017'
        }.get(x, x))


In [11]:
new_table = new_table[['ONS Partner Geography', 'Period','Flow','Pink Book Services', 'Seasonal Adjustment', 'Measure Type','Value','Unit','Marker' ]]